# Database management

> Classes and functions for managing access to the databases

In [1]:
#| default_exp db_mgmt

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export

from kedro.config import OmegaConfigLoader
from kedro.framework.project import settings
from pathlib import Path

import pandas as pd
import numpy as np
from tqdm import tqdm

import psycopg2



[02/11/25 13:46:50] INFO     Using                                                                  ]8;id=850623;file:///Users/moritzbeckmail.de/miniconda3/envs/if_sdk/lib/python3.11/site-packages/kedro/framework/project/__init__.py\__init__.py]8;;\:]8;id=932813;file:///Users/moritzbeckmail.de/miniconda3/envs/if_sdk/lib/python3.11/site-packages/kedro/framework/project/__init__.py#270\270]8;;\
                             '/Users/moritzbeckmail.de/miniconda3/envs/if_sdk/lib/python3.11/site-p                
                             ackages/kedro/framework/project/rich_logging.yml' as logging                          
                             configuration.                                                                        

In [4]:
#| export

def get_db_credentials():
    
    """
    Fetch PostgreSQL database credentials from the configuration file of the kedro project.

    Uses `OmegaConfigLoader` to load credentials stored under `credentials.postgres`.

    Returns:
        dict: A dictionary with the database connection details (e.g., host, port, user, password, dbname).
    """

    conf_path = str(Path(settings.CONF_SOURCE))
    conf_loader = OmegaConfigLoader(conf_source=conf_path)
    db_credentials = conf_loader["credentials"]["postgres"]

    return db_credentials

In [5]:
#| export

def insert_multi_rows(
    data_to_insert: pd.DataFrame,
    table_name: str,
    column_names: list,
    types: list,
    cur,
    conn,
    return_with_ids: bool = False,
    unique_columns: list = None,  # mandatory if return_with_ids is True
) -> pd.DataFrame | None:
    
    """
    Inserts data into the specified database table, with an optional return of database-assigned IDs.

    Args:
        data_to_insert (pd.DataFrame): DataFrame containing the data to be inserted.
        table_name (str): Name of the target database table.
        column_names (list): List of column names for the target table.
        types (list): List of Python types (e.g., [int, float]) for data conversion.
        cur (psycopg2.cursor): Database cursor for executing SQL commands.
        conn (psycopg2.connection): Database connection for committing transactions.
        return_with_ids (bool): If True, returns the original DataFrame with an additional "ID" column.

    Returns:
        pd.DataFrame | None: Original DataFrame with an "ID" column if `return_with_ids` is True; otherwise, None.
    """
    logger.info("-- in insert multi rows -- checking data")

    # Check for NaN values and log a warning if any are found
    if data_to_insert.isnull().values.any():
        logger.warning("There are NaNs in the data")
    
    # Ensure the DataFrame has the correct number of columns
    if len(column_names) != data_to_insert.shape[1]:
        raise ValueError("Number of column names does not match the number of columns in the DataFrame.")
    if len(types) != data_to_insert.shape[1]:
        raise ValueError("Number of types does not match the number of columns in the DataFrame.")
    
    logger.info("-- in insert multi rows -- converting data to list of tuples")
    # Convert to list of tuples and apply type casting

    data_values = data_to_insert.values.tolist()
    data_values = [tuple(typ(val) for typ, val in zip(types, row)) for row in data_values]
    
    logger.info("-- in insert multi rows -- preparing SQL")
    # Create SQL placeholders and query
    placeholders = ", ".join(["%s"] * len(column_names))
    column_names_str = ", ".join(f'"{col}"' for col in column_names)
    

    batch_size_for_commit = 1_000_000  # Adjust this based on your dataset size and transaction tolerance
    row_count = 0

    if return_with_ids:
        if not unique_columns:
            raise ValueError("unique_columns must be provided when return_with_ids is True")

        unique_columns_str = ", ".join(f'"{col}"' for col in unique_columns)
        insert_query = f"""
            INSERT INTO {table_name} ({column_names_str})
            VALUES ({placeholders})
            ON CONFLICT ({unique_columns_str})
            DO UPDATE SET "{unique_columns[0]}" = EXCLUDED."{unique_columns[0]}"
            RETURNING "ID";
        """
        ids = []

        
        
        # Insert row by row and collect IDs
        with tqdm(total=len(data_values), desc="Inserting rows") as pbar:
            for row in data_values:
                cur.execute(insert_query, row)
                row_id = cur.fetchone()
                if row_id:
                    ids.append(row_id[0])
                row_count += 1
                pbar.update(1)  # Update progress bar for each row
                
                # Commit every batch_size_for_commit rows
                if row_count % batch_size_for_commit == 0:
                    conn.commit()  # Commit the transaction
        conn.commit() 
        
        # Add IDs back to the original DataFrame
        data_with_ids = data_to_insert.copy()
        data_with_ids["ID"] = ids
        return data_with_ids

    else:
        insert_query = f"""
            INSERT INTO {table_name} ({column_names_str})
            VALUES ({placeholders})
            ON CONFLICT DO NOTHING;
        """
        
        # Insert row by row without returning IDs
        with tqdm(total=len(data_values), desc="Inserting rows") as pbar:
            for row in data_values:
                cur.execute(insert_query, row)
                row_count += 1
                pbar.update(1)  # Update progress bar for each row
                if row_count % batch_size_for_commit == 0:
                    conn.commit()  # Commit the transaction
                
        conn.commit()  # Commit all changes after processing

    return None

In [6]:
import psycopg2
from psycopg2.extras import execute_values


class SQLDatabase:
    def __init__(self, autocommit=True):
        self._credentials = get_db_credentials()["con"]
        self.connection = None
        self.autocommit = autocommit

    def connect(self):
        if not self.connection:
            self.connection = psycopg2.connect(self._credentials)
            self.connection.autocommit = self.autocommit

    def close(self):
        if self.connection:
            self.connection.close()
            self.connection = None

    def __enter__(self):
        self.connect()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type:
            self.connection.rollback()
        elif not self.autocommit:
            self.connection.commit()
        self.close()

    def execute_query(self, query: str, params: tuple = None, fetchall: bool = False, fetchone: bool = False, commit: bool = False):
        if fetchall and fetchone:
            raise ValueError("Both fetchall and fetchone cannot be True")
        if not self.connection:
            self.connect()
        with self.connection.cursor() as cur:
            cur.execute(query, params)
            result = cur.fetchall() if fetchall else cur.fetchone() if fetchone else None
        if commit and self.autocommit:
            self.connection.commit()
        return result

    def execute_multiple_queries(self, queries: list | str, params: list = None, fetchrows: bool = False, commit: bool = False):
        if not self.connection:
            self.connect()
        results = []
        with self.connection.cursor() as cur:
            if fetchrows:
                if isinstance(queries, str):
                    queries = [queries] * len(params)
                for query, par in zip(queries, params):
                    cur.execute(query, par)
                    results.append(cur.fetchone())
            else:
                if not isinstance(queries, str):
                    raise ValueError("For batch execution use a single query with multiple params (set fetchrows=True otherwise)")
                cur.executemany(queries, params)
        if commit and self.autocommit:
            self.connection.commit()
        return results if fetchrows else None

    def fetch_ids_bulk(self, table_name: str, id_column: str, column_names: list, rows: list[tuple]) -> list:
        """
        Retrieve IDs in one bulk query using the VALUES construct.
        """
        if not rows:
            return []
        # Build derived table with column names matching those in the main table.
        columns_str = ", ".join(column_names)
        join_clause = " AND ".join([f"t.{col} = v.{col}" for col in column_names])
        query = f"""
            SELECT t.{id_column}
            FROM {table_name} t
            JOIN (
                VALUES %s
            ) AS v({columns_str})
            ON {join_clause}
        """
        if not self.connection:
            self.connect()
        with self.connection.cursor() as cur:
            execute_values(cur, query, rows, page_size=100)
            results = cur.fetchall()
        return [int(r[0]) for r in results]


In [7]:
#| hide
import nbdev; nbdev.nbdev_export()

